In [ ]:
!pip install -q -U transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.9 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login() # Se te pedirá que introduzcas tu token de Hugging Face aquí


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id_gemma = "google/gemma-2b-it" # Gemma 2B Instruct
tokenizer_gemma = AutoTokenizer.from_pretrained(model_id_gemma)
model_gemma = AutoModelForCausalLM.from_pretrained(
    model_id_gemma,
    torch_dtype=torch.bfloat16, # O torch.float16 si tu GPU no soporta bfloat16
    device_map="auto"
)
print("Gemma-2b-it loaded successfully.")

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Gemma-2b-it loaded successfully.


In [ ]:
def get_gemma_response(full_prompt_text, temperature=0.1):
    """
    Envía un prompt a Gemma y devuelve la respuesta.
    full_prompt_text: El prompt completo a enviar.
    temperature: La temperatura para la generación.
    """
    # Gemma utiliza un formato de chat específico
    chat = [
        {"role": "user", "content": full_prompt_text},
    ]
    prompt_gemma = tokenizer_gemma.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer_gemma(prompt_gemma, return_tensors="pt").to(model_gemma.device)

    try:
        outputs = model_gemma.generate(
            **inputs,
            max_new_tokens=700,
            do_sample=True,
            temperature=temperature,
            top_p=0.9,
            pad_token_id=tokenizer_gemma.eos_token_id # Gemma a veces requiere esto
        )
        response_text = tokenizer_gemma.decode(outputs[0], skip_special_tokens=True)
        # Post-procesamiento para eliminar el prompt de la respuesta
        response_text = response_text[len(prompt_gemma):].strip()
        return response_text
    except Exception as e:
        print(f"Error calling Gemma: {e}")
        return "ERROR: Fallo al obtener respuesta de Gemma."

In [ ]:
# --- 20 Cláusulas Contractuales de Cesión de Derechos de Autor ---
clauses = [
    {"id": "clausula_01", "text": "El Autor cede a la Editorial el derecho exclusivo de reproducción de la Obra por un período de cinco (5) años en todo el territorio nacional español, con una compensación del 10% sobre las ventas netas."},
    {"id": "clausula_02", "text": "El Artista cede a la Galería el derecho no exclusivo de exhibición pública de la Obra visual por tiempo indefinido en la Unión Europea, sin percibir compensación económica directa."},
    {"id": "clausula_03", "text": "El Compositor cede a la Productora el derecho exclusivo de comunicación pública (incluida la radiodifusión) de la Composición por un período de tres (3) años en Estados Unidos de América, con un pago único de 5.000 euros."},
    {"id": "clausula_04", "text": "El Desarrollador cede a la Empresa el derecho exclusivo de distribución del Software en formato físico por diez (10) años a nivel mundial, recibiendo un 25% de los beneficios brutos."},
    {"id": "clausula_05", "text": "El Fotógrafo cede a la Agencia el derecho no exclusivo de reproducción y comunicación pública de las Imágenes por un período de un (1) año en territorio español, mediante una tarifa plana mensual de 200 euros."},
    {"id": "clausula_06", "text": "El Autor cede todos los derechos patrimoniales sobre la Obra por tiempo indefinido y en todo el universo, sin compensación alguna más allá de la mención de autoría."},
    {"id": "clausula_07", "text": "El Creador cede a la Plataforma el derecho no exclusivo de puesta a disposición interactiva de los Contenidos por dos (2) años para Latinoamérica, con una compensación del 50% de los ingresos publicitarios generados."},
    {"id": "clausula_08", "text": "El Diseñador cede a la Marca el derecho exclusivo de transformación y distribución del Diseño por quince (15) años en Europa y Norteamérica, a cambio de un pago inicial de 10.000 dólares."},
    {"id": "clausula_09", "text": "El Guionista cede a la Productora el derecho exclusivo de adaptación cinematográfica del Guion por siete (7) años en territorio mundial (con exclusión de China), con un porcentaje sobre la taquilla bruta del 1%."},
    {"id": "clausula_10", "text": "El Investigador cede a la Universidad el derecho no exclusivo de reproducción y comunicación pública de la Publicación por tiempo limitado a la vida del autor más 70 años en cualquier país con normativa de PI, sin remuneración específica."},
    {"id": "clausula_11", "text": "La Empresa XYZ cede a la Filial ABC el derecho exclusivo de reproducción de su material de marketing por tres (3) años en Alemania y Francia, con una compensación fija de 2.500 euros anuales."},
    {"id": "clausula_12", "text": "El Titular del Software cede a Licenciatario el derecho no exclusivo de distribución del Software a través de plataformas digitales por tiempo indefinido en Asia (excluyendo Japón), con un pago por cada descarga de 0,50 dólares."},
    {"id": "clausula_13", "text": "El Compositor A cede al Editor B el derecho exclusivo de sincronización de la Música por cuatro (4) años en territorio de la Unión Europea, mediante un adelanto de 1.000 libras y 50% de regalías."},
    {"id": "clausula_14", "text": "El Creador de Contenido cede a la Red Social el derecho no exclusivo de comunicación pública y transformación de sus publicaciones por tiempo indefinido en todo el mundo, sin compensación adicional por su uso."},
    {"id": "clausula_15", "text": "La Cía. Farmacéutica cede a la Empresa de Biotecnología el derecho exclusivo de reproducción de sus resultados de investigación por quince (15) años en América Latina, con una licencia fija de 50.000 dólares."},
    {"id": "clausula_16", "text": "El Podcast Host cede a la Distribuidora el derecho no exclusivo de distribución y puesta a disposición de los Episodios por diez (10) años en Canadá y Australia, a cambio de un porcentaje del 5% de los ingresos por suscripciones."},
    {"id": "clausula_17", "text": "El Escultor cede a la Fundación el derecho de exhibición pública de la Obra escultórica por un período de veinte (20) años en museos específicos de Nueva York (EE. UU.), sin fines de lucro y sin compensación directa."},
    {"id": "clausula_18", "text": "La Universidad cede al Consorcio de Investigación el derecho no exclusivo de reproducción de artículos científicos para fines educativos por tiempo indefinido en cualquier país miembro de la OCDE, sin contraprestación económica."},
    {"id": "clausula_19", "text": "El Productor de Video cede a la Empresa de Medios el derecho exclusivo de comunicación pública y distribución del Contenido de Video por ocho (8) años en Estados Unidos y México, a cambio de un pago por capítulo emitido de 1.000 dólares."},
    {"id": "clausula_20", "text": "El Autor cede todos los derechos de explotación sobre su Novela (reproducción, distribución, comunicación pública y transformación) por toda la duración de los derechos según la LPI en todo el mundo, con un adelanto de 3.000 euros y 8% de royalties."}
]

# --- Prompt Base para Extracción de Parámetros ---
PROMPT_BASE = """
Eres un asistente jurídico especializado en Derecho de Propiedad Intelectual. Tu tarea es analizar la siguiente cláusula contractual de cesión de derechos de autor.

**Instrucciones Clave:**
1.  **Extrae** únicamente la información explícita o directamente deducible de la cláusula. No realices interpretaciones jurídicas complejas ni añadas información que no esté directamente contenida en el texto.
2.  **Identifica** los cuatro parámetros fundamentales de la cesión:
    * **Tipo de Derecho Cedido:** ¿Qué derechos específicos se mencionan (ej. reproducción, distribución, comunicación pública, transformación, exhibición, adaptación, etc.) o se hace referencia a un conjunto de derechos (ej. "todos los derechos patrimoniales", "derechos de explotación")?
    * **Duración de la Cesión:** ¿Cuál es el plazo temporal de la cesión (ej. número de años, "tiempo indefinido", "vida del autor + 70 años", "toda la duración de los derechos")?
    * **Ámbito Territorial:** ¿Cuál es la delimitación geográfica específica de la cesión (ej. país, región, mundial, universo)?
    * **Forma de Compensación:** ¿Cómo se remunera la cesión (ej. pago único, porcentaje de ventas/beneficios, tarifa mensual/anual, sin compensación)?
3.  **Formato de Salida:** Presenta la información en un formato claro de lista, y para cada parámetro, indica **explícitamente la parte del texto de la cláusula** que fundamenta tu extracción.

**Cláusula a Analizar:**
[TEXTO DE LA CLÁUSULA]

**Ejemplo de Formato de Salida Deseado:**
- Tipo de Derecho Cedido: [Respuesta] (Texto fuente: "[Fragmento exacto de la cláusula]")
- Duración de la Cesión: [Respuesta] (Texto fuente: "[Fragmento exacto de la cláusula]")
- Ámbito Territorial: [Respuesta] (Texto fuente: "[Fragmento exacto de la cláusula]")
- Forma de Compensación: [Respuesta] (Texto fuente: "[Fragmento exacto de la cláusula]")
"""

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Modelo a utilizar
model_id = "google/gemma-2b-it"

# Cargar el tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Cargar el modelo
# Usamos torch_dtype=torch.bfloat16 para optimizar el uso de memoria en GPUs compatibles,
# si tu GPU no lo soporta, puedes probar torch.float16.
# device_map="auto" distribuye el modelo automáticamente en las GPUs disponibles.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
print(f"Modelo {model_id} cargado con éxito en {model.device}.")

# --- Función de Inferencia para Gemma ---
def get_gemma_response(clause_text, prompt_template=PROMPT_BASE, temperature=0.1):
    """
    Envía un prompt a Gemma para la extracción de parámetros de una cláusula
    y devuelve la respuesta generada.

    Args:
        clause_text (str): El texto de la cláusula a analizar.
        prompt_template (str): La plantilla del prompt base.
        temperature (float): La temperatura para la generación (0.0 para determinismo, >0.0 para creatividad).

    Returns:
        str: La respuesta generada por el modelo.
    """
    full_prompt_text = prompt_template.replace("[TEXTO DE LA CLÁUSULA]", clause_text)

    # Gemma usa un formato de chat específico para prompts de instrucción
    chat = [{"role": "user", "content": full_prompt_text}]
    formatted_prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

    try:
        outputs = model.generate(
            **inputs,
            max_new_tokens=700,  # Ajusta según la longitud máxima esperada de la respuesta
            do_sample=True,      # Permite muestreo para temperatura > 0
            temperature=temperature,
            top_p=0.9,           # Para controlar la diversidad del muestreo
            eos_token_id=tokenizer.eos_token_id, # Asegura que el modelo se detenga al final del token
            pad_token_id=tokenizer.eos_token_id  # Evita errores con longitudes de secuencia
        )
        # Decodificar y limpiar la respuesta (eliminar el prompt del principio)
        response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # El apply_chat_template añade el prompt al principio, lo quitamos
        if response_text.startswith(formatted_prompt):
            response_text = response_text[len(formatted_prompt):].strip()
        return response_text
    except Exception as e:
        print(f"Error al obtener respuesta de Gemma para la cláusula: {clause_text[:50]}... Error: {e}")
        return "ERROR: Fallo al obtener respuesta de Gemma."

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Modelo google/gemma-2b-it cargado con éxito en cuda:0.


In [ ]:
import time
import json # Para guardar los resultados en un formato estructurado

results_gemma = []

print("Iniciando generación de respuestas con google/gemma-2b-it...")
for i, clause_data in enumerate(clauses):
    clause_id = clause_data["id"]
    clause_text = clause_data["text"]

    print(f"\n--- Procesando {clause_id} ({i+1}/{len(clauses)}) ---")

    # Generar respuesta de Gemma
    gemma_response = get_gemma_response(clause_text, temperature=0.1) # Usamos 0.1 para un equilibrio entre determinismo y fluidez

    # Almacenar el resultado
    results_gemma.append({
        "clause_id": clause_id,
        "clause_text": clause_text,
        "model_name": "google/gemma-2b-it",
        "prompt_used": PROMPT_BASE.replace("[TEXTO DE LA CLÁUSULA]", clause_text), # Guarda el prompt completo para trazabilidad
        "model_response": gemma_response
    })
    print(f"Respuesta de Gemma para {clause_id} generada.")
    # Opcional: Pausa para evitar saturar la GPU o la API si se hacen muchas llamadas
    # time.sleep(1)

print("\nGeneración de respuestas de google/gemma-2b-it completada.")

# --- Guardar los resultados ---
output_filename = "gemma_experiment_5_1_1_results.json"
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(results_gemma, f, ensure_ascii=False, indent=4)

print(f"Resultados guardados en {output_filename}")

# --- Mostrar un ejemplo de resultado (opcional) ---
if results_gemma:
    print("\n--- Ejemplo de un resultado ---")
    print(json.dumps(results_gemma[0], indent=4, ensure_ascii=False))

Iniciando generación de respuestas con google/gemma-2b-it...

--- Procesando clausula_01 (1/20) ---
Respuesta de Gemma para clausula_01 generada.

--- Procesando clausula_02 (2/20) ---
Respuesta de Gemma para clausula_02 generada.

--- Procesando clausula_03 (3/20) ---
Respuesta de Gemma para clausula_03 generada.

--- Procesando clausula_04 (4/20) ---
Respuesta de Gemma para clausula_04 generada.

--- Procesando clausula_05 (5/20) ---
Respuesta de Gemma para clausula_05 generada.

--- Procesando clausula_06 (6/20) ---
Respuesta de Gemma para clausula_06 generada.

--- Procesando clausula_07 (7/20) ---
Respuesta de Gemma para clausula_07 generada.

--- Procesando clausula_08 (8/20) ---
Respuesta de Gemma para clausula_08 generada.

--- Procesando clausula_09 (9/20) ---
Respuesta de Gemma para clausula_09 generada.

--- Procesando clausula_10 (10/20) ---
Respuesta de Gemma para clausula_10 generada.

--- Procesando clausula_11 (11/20) ---
Respuesta de Gemma para clausula_11 generada.

-

In [ ]:
import json
import os

# --- Configuración ---
json_filename = "gemma_experiment_5_1_1_results.json"
output_markdown_filename = "resultados_experimento_gemma_clausulas_PI.md"

# --- Paso 1: Cargar los resultados del archivo JSON ---
results_gemma = []
if os.path.exists(json_filename):
    with open(json_filename, 'r', encoding='utf-8') as f:
        results_gemma = json.load(f)
    print(f"[{len(results_gemma)} resultados cargados desde {json_filename}]")
else:
    print(f"Error: El archivo '{json_filename}' no se encontró. Asegúrate de que el experimento se haya ejecutado y el archivo exista.")
    # Si el archivo no existe, no hay nada que procesar. Podrías salir o manejar este caso.
    exit() # Salimos del script si no hay datos.

# --- Paso 2: Generar el contenido en formato Markdown ---
markdown_content = []

# Encabezado del documento Markdown
markdown_content.append("# Informe de Extracción de Parámetros de Cláusulas de Cesión de Derechos de Autor (Modelo Gemma)")
markdown_content.append("## Experimento 5.1.1: Recuperación de Cláusulas Contractuales de Cesión de Derechos de Autor")
markdown_content.append("---")
markdown_content.append("Este documento presenta las respuestas generadas por el modelo de Inteligencia Artificial **google/gemma-2b-it** al procesar una serie de cláusulas contractuales de cesión de derechos de autor. La tarea asignada al modelo fue la extracción de parámetros específicos (Tipo de Derecho Cedido, Duración, Ámbito Territorial y Forma de Compensación) sin realizar interpretaciones jurídicas complejas. Este informe facilita la revisión por parte de juristas para la posterior evaluación del *Recall* y el LIAS.\n")
markdown_content.append(f"**Fecha de generación del informe:** {time.strftime('%Y-%m-%d %H:%M:%S')}")
markdown_content.append(f"**Número total de cláusulas procesadas:** {len(results_gemma)}\n")
markdown_content.append("---")

# Iterar sobre cada resultado y formatearlo para Markdown
for entry in results_gemma:
    clause_id = entry.get("clause_id", "ID Desconocido")
    clause_text = entry.get("clause_text", "Texto de cláusula no disponible.")
    model_name = entry.get("model_name", "Modelo Desconocido")
    model_response = entry.get("model_response", "Respuesta del modelo no disponible.")
    # prompt_used = entry.get("prompt_used", "Prompt no disponible.") # Puedes incluir el prompt si lo deseas

    markdown_content.append(f"\n## Cláusula: {clause_id}\n")
    markdown_content.append("### Texto Original de la Cláusula:\n")
    markdown_content.append(f"```\n{clause_text}\n```\n") # Usar bloque de código para el texto original

    markdown_content.append(f"### Respuesta del Modelo ({model_name}):\n")
    markdown_content.append(f"```\n{model_response}\n```\n") # Usar bloque de código para la respuesta del modelo

    # Opcional: Si quieres incluir el prompt completo para cada cláusula
    # markdown_content.append("#### Prompt Enviado al Modelo:\n")
    # markdown_content.append(f"```\n{prompt_used}\n```\n")

    markdown_content.append("---\n") # Separador entre cláusulas

# --- Paso 3: Guardar el contenido Markdown en un archivo ---
try:
    with open(output_markdown_filename, 'w', encoding='utf-8') as f:
        f.write("\n".join(markdown_content))
    print(f"[{len(results_gemma)} cláusulas formateadas. Archivo '{output_markdown_filename}' generado con éxito.]")
except Exception as e:
    print(f"Error al escribir el archivo Markdown: {e}")

[20 resultados cargados desde gemma_experiment_5_1_1_results.json]
[20 cláusulas formateadas. Archivo 'resultados_experimento_gemma_clausulas_PI.md' generado con éxito.]
